In [305]:
import numpy as np
import scipy.stats as ss
np.set_printoptions(precision=2, suppress=True)
rng = np.random.default_rng(1)
np.random.seed(1)

GBM :

$dS_t = S_t \mu dt + S_t \sigma W_t, \qquad W_t \sim N(0, dt)$

$S_T = S_0 \times e^{(\mu - \frac{\sigma^2}{2}) T + \sigma \sqrt{T} Z} $

In [306]:
def GBM(S0, mu, sigma, T):
    np.random.seed(42)
    Z = np.random.standard_normal(1)
    S_T = S0 * np.exp((mu - sigma**2 / 2)*T + sigma * np.sqrt(T) * Z)
    return S_T

S0 = 125
mu = 0.0825
sigma = 0.42
T = 1/255

GBM(S0, mu, sigma, T) - S0

array([1.64])

In [307]:
def BS_price(S0, K, r, sigma, T, t, call_or_put="C"):
    T = T-t
    d1 = (np.log(S0/K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
    print("d1 =", d1)
    d2 = d1 - sigma * np.sqrt(T)
    
    if call_or_put == "C":
        return S0 * ss.norm.cdf(d1) - np.exp(-r*T) * K * ss.norm.cdf(d2)
    else:
        return np.exp(-r*T) * K * ss.norm.cdf(-d2) - S0 * ss.norm.cdf(-d1)

In [308]:
BS_price(12.5, 15, 0.01, 0.45, 1, 0, "P")

d1 = -0.15793679287545465


3.7827324909512487

In [309]:
d1 = -0.15793679287545465
N_d1 = ss.norm.cdf(-0.15793679287545465)
N_d1 - 1

-0.5627466973133798

In [310]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import lognorm, norm
import scipy.stats as ss

np.set_printoptions(precision=4, suppress=True)

def BS_price(S0, K, r, sigma, T, t, call_or_put="C"):
    T = T-t
    d1 = (np.log(S0/K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if call_or_put == "C":
        rho = K * (T-t) * np.exp(-r * (T-t)) * ss.norm.cdf(d2)
        print("rho of the call option =", rho)
        return S0 * ss.norm.cdf(d1) - np.exp(-r*T) * K * ss.norm.cdf(d2)
    else:
        return np.exp(-r*T) * K * ss.norm.cdf(-d2) - S0 * ss.norm.cdf(-d1)
    
def MC_price(S0, K, r, sigma, T, N, M, t, call_or_put="C"):
    np.random.seed(42)
    T = T - t
    dt = T / N
    S = np.zeros((N+1, M))
    S[0] = S0  # at t=0, all prices are S0. (first row is S0 repeated.)
    rn = np.random.standard_normal((N, M))
    multipliers = np.exp((r - sigma**2 / 2) * dt + sigma * np.sqrt(dt) * rn)
    # print("multipliers.cumprod() =\n", multipliers.cumprod(axis=0))
    S[1:, :] = S0 * multipliers.cumprod(axis=0)  # price paths are generated
    # print("S =\n", S)
    
    if call_or_put == "C":
        average_payoff = np.maximum(S[-1, :] - K, 0).mean()
    else:
        average_payoff = np.maximum(K - S[-1, :], 0).mean()
    
    opt_price = np.exp(-r*dt) * average_payoff
    return opt_price

In [311]:
K = 23
S0 = 22.75
r = 0.02
T = 7/12
sigma = 0.45

BS_price(S0, K, r, sigma, T, 0, "C")

3.1113021537768724

In [312]:
MC_price(S0, K, r, sigma, T, 2500, 2500, 0, "C")

3.175239308600649

In [313]:
K = 122
S0 = 118.75
r = 0.015
T = 18/12
sigma = 0.25

call_price_BS = BS_price(S0, K, r, sigma, T, 0, "C")
print("call_price_BS BS =", call_price_BS)

stop_num = 5000
# M_arr = np.arange(start=100, stop=stop_num, step=200)
# put_price_MC_arr = np.array([MC_price(S0, K, r, sigma, T, 1000, M, t=0, call_or_put="C") for M in M_arr])
call_price_MC_arr = MC_price(S0, K, r, sigma, T, N=365, M = stop_num, t=0, call_or_put="C")
print("call_price_MC_arr =", call_price_MC_arr)

# plt.plot(M_arr, (put_price_MC_arr - put_price_BS))
# plt.hlines(y=0.2, xmin=0, xmax=stop_num)
# plt.hlines(y=-0.2, xmin=0, xmax=stop_num)

print("MC - BS =", call_price_MC_arr - call_price_BS)

call_price_BS BS = 14.215238492284897
call_price_MC_arr = 13.8716594145513
MC - BS = -0.3435790777335974


In [314]:
def vasicek(r0, k, theta, sigma, T, N, M):
    """_summary_

    Args:
        r0 (_type_): starting interest rate
        k (_type_): speed of mean-reversion
        theta (_type_): long term average of interest rates
        sigma (_type_): short-term volatility of interest rates
        T (_type_): maturity
        N (_type_): num of steps
        M (_type_): num of paths
    """
    np.random.seed(0)
    dt = T / N
    rn = np.random.standard_normal((N, M))
    R = np.zeros((N, M))
    R[0, :] = r0  # first row is initial rates
    # print("R =\n", R)
    for i in range(N-1):
        # print("R[i, :] =\n", R[i, :])
        dr = k * (theta - R[i, :]) * dt + sigma * np.sqrt(dt) * rn[i, :]
        R[i+1, :] = R[i, :] + dr
    return R

M = 1  # Number of paths for MC
N = 125  # Number of steps
T = 3/12  # Maturity
r0 = 0.985
k = 0.22
theta = 0.018
sigma = 1.75

rates = vasicek(r0, k, theta, sigma, T, N, M)
rates_min = rates[-1, :].min()
rates_max = rates[-1, :].max()

print("rates_mean =", rates[-1].mean())
print("rates_min =", rates_min)
print("rates_max =", rates_max)

rates_mean = 2.2228553177248997
rates_min = 2.2228553177248997
rates_max = 2.2228553177248997


In [315]:
def vasicek(r0, K, theta, sigma, T, N, M):
    np.random.seed(42)
    dt = T / N
    rates = np.zeros((N, M))
    rates[0, :] = r0
    for j in range(M):
        for i in range(1, N):
            dr = (
                K * (theta - rates[i - 1, j]) * dt
                + sigma * np.sqrt(dt) * np.random.normal()
            )
            rates[i, j] = rates[i - 1, j] + dr
    return rates

rates = vasicek(r0, k, theta, sigma, T, N, M)
rates[-1,  :].mean()

0.2112951078387043

In [316]:
S0 = 32.5
K = 27.5
T = 4/12
sigma = 0.45
r = 0.0275
t = 0

BS_price(S0, K, r, sigma, T, t, "C")

6.391440240040197